<a href="https://colab.research.google.com/github/jswooo/HuggingFace/blob/main/2-1(behind_pipeline).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tenso

In [ ]:
from transformers import pipeline, AutoTokenizer

In [ ]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english" # 기본 
tokenizer =  AutoTokenizer.from_pretrained(checkpoint) # checkpoint 모델의 tokenizer data를 fetch 

In [ ]:
raw_inputs = [
    "Learning 'natural language processing' is so hard",
    "I'm looking forward to participating in a competition"
]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 101, 4083, 1005, 3019, 2653, 6364, 1005, 2003, 2061, 2524,  102,    0],
        [ 101, 1045, 1005, 1049, 2559, 2830, 2000, 8019, 1999, 1037, 2971,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
from transformers import AutoModel

# pretrain model을 다운로드 
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# 결과 
outputs = model(**inputs)
print(outputs['last_hidden_state'].shape)

torch.Size([2, 12, 768])


In [ ]:
from transformers import AutoModelForSequenceClassification

# 감성 분석을 위해서 automodelfor~ 씀 
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

print(outputs.logits.shape)
print(outputs.logits)

torch.Size([2, 2])
tensor([[ 3.7753, -3.1374],
        [-3.6063,  3.8339]], grad_fn=<AddmmBackward0>)


In [ ]:
# softmax function으로 확률 출력 

import torch 

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[9.9901e-01, 9.9412e-04],
        [5.8680e-04, 9.9941e-01]], grad_fn=<SoftmaxBackward0>)


In [ ]:
# 어떤게 positive인지.. 
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}